## Bitcoin data crawling

In [1]:
import requests
import pandas as pd

def get_binance_klines(symbol="BTCUSDT", interval="1h", limit=1000):
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    data = requests.get(url, params=params).json()
    
    cols = ['Open time','Open','High','Low','Close','Volume',
            'Close time','Quote asset volume','Trades',
            'Taker buy base','Taker buy quote','Ignore']
    df = pd.DataFrame(data, columns=cols)
    df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
    df = df[['Open time','Open','High','Low','Close','Volume']]
    
    # Convert numeric columns to float
    numeric_cols = ['Open','High','Low','Close','Volume']
    df[numeric_cols] = df[numeric_cols].astype(float)
    
    
    return df

df_bitcoin = get_binance_klines()
print(df_bitcoin.head())


            Open time       Open       High        Low      Close      Volume
0 2025-10-22 01:00:00  107986.15  108460.02  107986.14  108364.21  1493.26761
1 2025-10-22 02:00:00  108364.22  108678.21  108198.85  108198.85   699.65823
2 2025-10-22 03:00:00  108198.85  108219.12  107860.92  108216.02  1121.03699
3 2025-10-22 04:00:00  108216.03  108635.92  108162.09  108625.60   696.45277
4 2025-10-22 05:00:00  108625.61  108638.99  107879.93  108166.81  1434.44211


In [2]:
df_bitcoin.dtypes

Open time    datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Volume              float64
dtype: object

In [3]:
df_bitcoin.to_csv("Data/bitcoin_data.csv", index=False)

In [15]:
df_bitcoin.head()

,Open time,Open,High,Low,Close,Volume
0,2025-10-22 01:00:00,107986.15,108460.02,107986.14,108364.21,1493.26761
1,2025-10-22 02:00:00,108364.22,108678.21,108198.85,108198.85,699.65823
2,2025-10-22 03:00:00,108198.85,108219.12,107860.92,108216.02,1121.03699
3,2025-10-22 04:00:00,108216.03,108635.92,108162.09,108625.60,696.45277
4,2025-10-22 05:00:00,108625.61,108638.99,107879.93,108166.81,1434.44211


In [ ]:
import requests
import pandas as pd
import yfinance as yf
import time
import os

# --- PART 1: BINANCE CRAWLER (CRYPTO) ---
def get_binance_klines(symbol="BTCUSDT", interval="1h", limit=1000):
    print(f"🚀 Fetching {symbol} from Binance...")
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status() # Check for API errors
        data = response.json()
        
        cols = ['Open time','Open','High','Low','Close','Volume',
                'Close time','Quote asset volume','Trades',
                'Taker buy base','Taker buy quote','Ignore']
        df = pd.DataFrame(data, columns=cols)
        
        # Convert Timestamp to Datetime
        df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
        
        # Select and Convert numeric columns
        df = df[['Open time','Open','High','Low','Close','Volume']]
        numeric_cols = ['Open','High','Low','Close','Volume']
        df[numeric_cols] = df[numeric_cols].astype(float)
        
        # Set the datetime as the Index so we can join with Yahoo data
        df = df.set_index('Open time')
        
        # Rename columns to identify them as Bitcoin
        df = df.rename(columns={
            'Close': 'BTC_Close',
            'Volume': 'BTC_Volume',
            'Open': 'BTC_Open',
            'High': 'BTC_High',
            'Low': 'BTC_Low'
        })
        
        return df
    except Exception as e:
        print(f"❌ Error fetching from Binance: {e}")
        return None

# --- PART 2: YAHOO FINANCE CRAWLER (MACRO) ---
TICKERS_CONFIG = {
    "SP500_Fut": "ES=F", # S&P 500 Futures
    "DXY": "DX-Y.NYB",   # Dollar Index 
    "VIX": "^VIX",       # Volatility Index
    "Gold": "GC=F"       # Gold Futures
}

def download_and_clean(name, ticker, period="2mo", interval="1h"):
    print(f"⬇️ Downloading {name} ({ticker})...")
    try:
        # progress=False hides the yahoo loading bar to keep terminal clean
        df = yf.download(tickers=ticker, period=period, interval=interval, progress=False)
        
        if df.empty:
            print(f"⚠️ Warning: No data found for {name}")
            return None 
            
        # Flatten HEADER if MultiIndex exists
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        
        df.columns.name = None # Remove "Price" label
        
        # Handle Timezone (Remove timezone to match Binance UTC)
        if df.index.tz is not None: 
            df.index = df.index.tz_localize(None)
        
        # Choosing Columns and Rename
        df = df[['Close']]
        df = df.rename(columns={'Close': f'{name}_Close'})
        
        return df 
    except Exception as e:
        print(f"❌ Error while downloading {name}: {e}")
        return None 

# --- PART 3: MAIN EXECUTION ---
def main_crawl_process():
    print("--- STARTING DATA PIPELINE ---\n")
    
    # 1. Get Master Data (Bitcoin)
    df_bitcoin = get_binance_klines()
    
    if df_bitcoin is None or df_bitcoin.empty:
        print("Error: Bitcoin Historical Data is not available. Stopping.")
        return 

    master_df = df_bitcoin
    
    # 2. Get Macro Data and Merge
    for name, ticker in TICKERS_CONFIG.items():
        df_macro = download_and_clean(name, ticker)
        
        if df_macro is not None:
            # Join based on Time Index (Left Join keeps all Bitcoin rows)
            master_df = master_df.join(df_macro, how="left")
            time.sleep(1) # Be polite to API
    
    # 3. Handle Missing Values (Forward Fill for weekends)
    print("\n Processing Missing Values...")
    nan_before = master_df.isna().sum().sum()
    
    master_df = master_df.ffill() # Forward fill (Sat/Sun gets Friday price)
    master_df = master_df.bfill() # Backward fill (for start of data)
    
    nan_after = master_df.isna().sum().sum()
    print(f"   -> Filled {nan_before - nan_after} missing values.")

    # 4. Save to CSV
    # Check if 'Data' folder exists, if not create it
    if not os.path.exists('Data'):
        os.makedirs('Data')
        
    master_df.to_csv("Data/market_data.csv")
    print(f"\n✅ SUCCESS! Data saved to 'Data/market_data.csv'")
    print(f" Shape: {master_df.shape}")
    print(master_df.head())
    
    return master_df

if __name__ == "__main__":
    main_crawl_process()

--- STARTING DATA PIPELINE ---

🚀 Fetching BTCUSDT from Binance...
⬇️ Downloading SP500_Fut (ES=F)...


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21276\552420553.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=ticker, period=period, interval=interval, progress=False)


⬇️ Downloading DXY (DX-Y.NYB)...


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21276\552420553.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=ticker, period=period, interval=interval, progress=False)


⬇️ Downloading VIX (^VIX)...


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21276\552420553.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=ticker, period=period, interval=interval, progress=False)


⬇️ Downloading Gold (GC=F)...


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21276\552420553.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=ticker, period=period, interval=interval, progress=False)



 Processing Missing Values...
   -> Filled 1646 missing values.

✅ SUCCESS! Data saved to 'Data/market_data.csv'
 Shape: (1000, 9)
                      BTC_Open   BTC_High    BTC_Low  BTC_Close  BTC_Volume  \
Open time                                                                     
2025-10-22 03:00:00  108198.85  108219.12  107860.92  108216.02  1121.03699   
2025-10-22 04:00:00  108216.03  108635.92  108162.09  108625.60   696.45277   
2025-10-22 05:00:00  108625.61  108638.99  107879.93  108166.81  1434.44211   
2025-10-22 06:00:00  108166.82  108331.75  107945.50  107980.02   675.25677   
2025-10-22 07:00:00  107980.02  108321.64  107610.00  108321.63  1077.00799   

                     SP500_Fut_Close  DXY_Close  VIX_Close   Gold_Close  
Open time                                                                
2025-10-22 03:00:00          6779.75  98.898003      17.77  4143.700195  
2025-10-22 04:00:00          6784.75  98.884003      17.77  4141.899902  
2025-10-22 05:00:0